This notebook demonstrates how to create a simple Python chatbot using Tkinter for the GUI and Sentence Transformers for natural language processing (NLP). The chatbot has two modes: Answer Mode and Quiz Mode.




### **1. Importing Required Libraries**
First, we import the necessary libraries:

- **tkinter** for the GUI.

- **json** to load the dataset.

- **torch** for tensor operations.

- **SentenceTransformer** for NLP embeddings.

- **random** for selecting random quiz questions.



In [1]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import json
import torch
import random
from sentence_transformers import SentenceTransformer, util
from datetime import datetime
import time

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **2. Loading the Dataset**
The dataset is stored in a JSON file (data.json). It contains two sections: answer_mode and quiz_mode.

- **Answer Mode**: Contains questions and their corresponding answers.

- **Quiz Mode**: Contains quiz questions, options, and correct answers.

In [2]:
# Load dataset
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Prepare Answer Mode dataset
answer_data = {item["question"]: item["answer"] for item in data["answer_mode"]}
questions = list(answer_data.keys())
answers = list(answer_data.values())

### **3. Loading the Sentence Transformer Model**
We use the **all-MiniLM-L6-v2** model from Sentence Transformers to encode questions into embeddings. This allows us to compare the similarity between user input and predefined questions.

In [3]:
# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode all questions into embeddings
question_embeddings = model.encode(questions, convert_to_tensor=True)

### **4. Function to Get the Best-Matching Answer**
This function calculates the cosine similarity between the user's input and the predefined questions. If the similarity score is above a threshold (0.5), it returns the best-matching answer.

In [5]:
# Function to get the best-matching answer using NLP
def get_best_answer(user_question):
    user_embedding = model.encode(user_question, convert_to_tensor=True)
    cos_scores = util.cos_sim(user_embedding, question_embeddings)[0]
    best_match_idx = torch.argmax(cos_scores).item()
    best_score = cos_scores[best_match_idx].item()

    if best_score > 0.5:  # Threshold for similarity
        return answers[best_match_idx]
    return "Sorry, I don't know the answer to that question."

### **5. Preparing Quiz Mode Data**
The quiz mode data is loaded from the JSON file. It contains questions, options, and correct answers.

In [6]:
# Prepare Quiz Mode dataset
quiz_mode_data = data["quiz_mode"]
mode = "answer"
current_question = None
correct_answer = None
score = 0
total_questions = 0

### **6. Function to Get Timestamp**
This function returns the current time in HH:MM:SS format, which is used to timestamp messages in the chat.

In [7]:
# Function to get timestamp
def get_timestamp():
    return datetime.now().strftime("%H:%M:%S")  # Example: 14:30:15

### **7. Function to Switch Modes**
The chatbot can switch between Answer Mode and Quiz Mode. This function toggles the mode and updates the GUI.

In [8]:
# Function to switch modes
def switch_mode():
    global mode
    mode = "quiz" if mode == "answer" else "answer"
    mode_label.config(text=f"Mode: {mode.capitalize()} Mode")
    chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: Switched to {mode.capitalize()} Mode.\n", "bot")
    chat_area.yview(tk.END)

### **8. Function to Handle User Input**
This function processes user input based on the current mode:

- **Answer Mode**: The chatbot responds with the best-matching answer.

- **Quiz Mode**: The chatbot presents a quiz question and checks the user's answer.

In [9]:
# Function to handle user input
def handle_input(event=None):  # Add event argument for Enter key support
    global current_question, correct_answer, score, total_questions
    user_input = user_entry.get().strip()

    if not user_input:
        return

    chat_area.insert(tk.END, f"\n[{get_timestamp()}] [You]: {user_input}\n", "user")
    user_entry.delete(0, tk.END)

    if mode == "answer":
        if user_input.lower() in ["exit", "quit", "bye"]:
            chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: Goodbye! Have a nice day.\n", "bot")
            root.after(1000, root.destroy)  # Close the window after 1 second
        else:
            response = get_best_answer(user_input)
            chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: {response}\n", "bot")

    elif mode == "quiz":
        if current_question:
            if user_input.lower() == correct_answer.lower():
                chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: ✅ Correct!\n", "bot")
                score += 1
            else:
                chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: ❌ Wrong! The correct answer was: {correct_answer}\n", "bot")

            total_questions += 1
            chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: Score: {score}/{total_questions}\n", "bot")
            current_question = None
            correct_answer = None
        else:
            question_data = random.choice(quiz_mode_data)
            current_question = question_data["question"]
            correct_answer = question_data["correct_answer"]
            options = "\n".join(f"- {opt}" for opt in question_data["options"])
            chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: {current_question}\nOptions:\n{options}\n", "bot")

    chat_area.yview(tk.END)

### **9. Function to Save Chat History**
This function saves the chat history to a text file with a timestamp.

In [10]:
# Function to save chat history
def save_chat():
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")  # Format: YYYY-MM-DD_HH-MM-SS
    filename = f"chat_history_{timestamp}.txt"  # Unique filename with timestamp
    with open(filename, "w", encoding="utf-8") as file:
        file.write(chat_area.get("1.0", tk.END))
    chat_area.insert(tk.END, f"\n[{get_timestamp()}] [Bot]: Chat saved as 'chat_history.txt'!\n", "bot")

### **10. GUI Setup**

The GUI is built using Tkinter. It includes:

A chat display area.

An input field for the user.

Buttons to send messages, switch modes, and save the chat.

In [12]:
# GUI Setup
root = tk.Tk()
root.title("Python Chatbot")
root.geometry("600x600")
root.configure(bg="#2C2F33")

# Chat display area (Styled)
chat_frame = ttk.Frame(root, padding=10)
chat_frame.pack(pady=10, fill="both", expand=True)

chat_area = scrolledtext.ScrolledText(chat_frame, wrap=tk.WORD, width=70, height=20, bg="#23272A", fg="white", font=("Arial", 12), bd=0, relief=tk.FLAT)
chat_area.pack(padx=5, pady=5, fill="both", expand=True)
chat_area.insert(tk.END, "[Bot]: Hello! Ask me anything about Python or switch to Quiz Mode.\n", "bot")

# User input area
input_frame = ttk.Frame(root, padding=10)
input_frame.pack(fill="x", pady=5)

user_entry = ttk.Entry(input_frame, width=50, font=("Arial", 12))
user_entry.pack(side=tk.LEFT, padx=5, pady=5, fill="x", expand=True)
user_entry.bind("<Return>", handle_input)  # Press Enter to send message

# Button Styling
style = ttk.Style()
style.configure("TButton", font=("Arial", 11), padding=6, relief="raised", borderwidth=3)
style.configure("Send.TButton", background="#4CAF50" )  # Green for Send
style.map("Send.TButton", background=[("active", "#388E3C")])
style.configure("Mode.TButton", background="#FF9800")  # Orange for Mode
style.map("Mode.TButton", background=[("active", "#F57C00")])

# Buttons
send_button = ttk.Button(input_frame, text="Send", command=handle_input, style="Send.TButton")
send_button.pack(side=tk.RIGHT, padx=5)

mode_frame = ttk.Frame(root, padding=5)
mode_frame.pack(fill="x")

mode_label = ttk.Label(mode_frame, text="Mode: Answer Mode", font=("Arial", 12, "bold"), background="#2C2F33", foreground="white")
mode_label.pack(side=tk.LEFT, padx=10)

mode_button = ttk.Button(mode_frame, text="Switch Mode", command=switch_mode, style="Mode.TButton")
mode_button.pack(side=tk.RIGHT, padx=10)

save_button = ttk.Button(mode_frame, text="Save Chat", command=save_chat, style="TButton")
save_button.pack(side=tk.RIGHT, padx=10)

# Chat Text Colors
chat_area.tag_config("bot", foreground="#7289DA")  # Blue for bot
chat_area.tag_config("user", foreground="#43B581")  # Green for user

# Run GUI
root.mainloop()

### **11. Running the Chatbot**
To run the chatbot, execute all the cells in the notebook. The Tkinter window will open, and you can interact with the chatbot.